In [ ]:
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import argparse
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import matplotlib.gridspec as gridspec
from matplotlib.colorbar import Colorbar
from matplotlib.colors import Normalize, TwoSlopeNorm
import glob
import os

In [ ]:
dir = '/Users/smurugan9/research/aislens/aislens_emulation/data/external/SORRMv2.1.ISMF/regridded_output/'
dir = '/Users/smurugan9/research/aislens/MALI-ismip6-0721/hist_08/output/'

'floatingBasalMassBalApplied'

In [ ]:
tmpdir = '/Users/smurugan9/research/aislens/aislens_emulation/data/interim/MALI_runs/ismip6-1108/hist04/globalStats.nc'
tmpds = xr.open_dataset(tmpdir)

In [ ]:
hist = '/Users/smurugan9/research/aislens/aislens_emulation/data/interim/MALI_runs/ismip6-1108/hist04/'
ctrl = '/Users/smurugan9/research/aislens/aislens_emulation/data/interim/MALI_runs/ismip6-1108/ctrlAE_04/'
exp01 = '/Users/smurugan9/research/aislens/aislens_emulation/data/interim/MALI_runs/ismip6-1108/expAE01_04/'

In [ ]:
globalStats = 'globalStats.nc'
regionalStats = 'regionalStats.nc'

In [ ]:
ctrlRegional = xr.open_dataset(ctrl+regionalStats)
exp01Regional = xr.open_dataset(exp01+regionalStats)

#ctrlglobal = xr.open_dataset(ctrl+globalStats)
exp01global = xr.open_dataset(exp01+globalStats)

In [ ]:
exp01global.surfaceSpeedMax.plot()

In [ ]:
dir1 = '/Users/smurugan9/research/aislens/aislens_emulation/data/interim/MALI_runs/ismip6_obs23_IC/globalStats.nc'
dir2 = '/Users/smurugan9/research/aislens/aislens_emulation/data/interim/MALI_runs/ismip6-relax/globalStats.nc'
dir3 = '/Users/smurugan9/research/aislens/aislens_emulation/data/interim/MALI_runs/ismip6-1101-obs23/globalStats.nc'

In [ ]:
ds = xr.open_dataset(dir1)
ds2 = xr.open_dataset(dir2)
ds3 = xr.open_dataset(dir3)
ds.deltat[:50].plot(label='obs23MALI (50 of 325 timesteps)')
ds2.deltat.plot(label='origMALI')
tmpds.deltat.plot(label='obs23MALI (Modified timestep)')
#ds3.deltat[:50].plot(label='obs23-1101-MALI')
plt.title('ismip6/hist04 runs')
plt.legend()

In [ ]:
#ds.surfaceSpeedMax[:50].plot(label='obs23MALI (100 of 325 timesteps)')
ds2.surfaceSpeedMax.plot(label='origMALI')
tmpds.surfaceSpeedMax.plot(label='obs23MALI (Modified timestep)')
#ds3.deltat[:50].plot(label='obs23-1101-MALI')
plt.title('ismip6/hist04 runs')
plt.legend()

In [ ]:
MELTDRAFT_OBS = xr.open_dataset(main_dir / DIR_basalMeltObs / FILE_MeltDraftObs, chunks={'x':729, 'y':729})

In [ ]:
MELTDRAFT_OBS.rio.write_crs("epsg:3031",inplace=True);


In [ ]:
h = MELTDRAFT_OBS.draft
if 'time' in h.dims:
    tdim = 'time'
elif 'Time' in h.dims:
    tdim = 'Time'

h_mean = h.mean(tdim)

IMBIEregions = range(6,33)
iceShelfRegions = range(33,133)

In [ ]:
mlt = MELTDRAFT_OBS.melt.rio.clip(icems.loc[[34],'geometry'].apply(mapping),icems.crs,drop=False)
mlt_mean = mlt.mean(tdim)
# Dedraft: Linear Regression with SSH over chosen basin
print('calculating linear regression for catchment {}'.format(icems.name.values[i]))
mlt_rgrs = xr_linregress(h, mlt_mean, dim=tdim) # h = independent variable

In [ ]:
mlt_prd = mlt_rgrs.slope*h_mean + mlt_rgrs.intercept

In [ ]:
icems.name.values

In [ ]:
DIR_external = 'data/external/'
DIR_interim = 'data/interim/'

# DATASET FILEPATHS
# Ocean model output - E3SM (SORRMv2.1.ISMF), data received from Darin Comeau / Matt Hoffman at LANL
DIR_SORRMv21 = 'data/external/SORRMv2.1.ISMF/regridded_output/'
FILE_SORRMv21 = 'Regridded_SORRMv2.1.ISMF.FULL.nc'
FILE_SORRMv21_DETRENDED_TS = 'SORRMv21_detrended_ts.nc'

ds = xr.open_dataset(main_dir / DIR_SORRMv21 / FILE_SORRMv21_DETRENDED_TS)


In [ ]:
ds["flux_sum"] = ds.__xarray_dataarray_variable__

In [ ]:
plt.figure(figsize=(25,8))
ds.__xarray_dataarray_variable__.plot()

In [ ]:
ds.__xarray_dataarray_variable__.rename("flux_xy_sum")

In [ ]:
ds